## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-22-14-53-19 +0000,bbc,UN's top court says Israel obliged to allow UN...,https://www.bbc.com/news/articles/c17pqxzl0yzo...
1,2025-10-22-14-52-56 +0000,startribune,Inside Luigi Mangione’s missing months,https://www.startribune.com/inside-luigi-mangi...
2,2025-10-22-14-52-42 +0000,nypost,Alarming video shows knife-wielding maniac rep...,https://nypost.com/2025/10/22/us-news/knife-wi...
3,2025-10-22-14-44-27 +0000,nyt,"In First Six Months, Cost of Weather Catastrop...",https://www.nytimes.com/2025/10/22/climate/tru...
4,2025-10-22-14-41-43 +0000,cbc,Some Conservative MPs doubt Poilievre’s leader...,https://www.cbc.ca/news/politics/politics-poil...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
180,trump,49
3,israel,14
560,hamas,12
7,gaza,12
74,new,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
94,2025-10-22-10-00-00 +0000,nypost,Sen. John Kennedy talks Trump’s social media h...,https://nypost.com/2025/10/22/us-news/sen-john...,99
249,2025-10-21-21-57-30 +0000,bbc,Trump says he did not want 'wasted meeting' af...,https://www.bbc.com/news/articles/c4gjp73gp41o...,93
328,2025-10-21-17-39-21 +0000,nyt,Latest Trump-Zelensky Meeting Yields No Progre...,https://www.nytimes.com/2025/10/21/world/europ...,92
337,2025-10-21-17-10-44 +0000,nypost,Trump’s religious-freedom envoy pick visits Mi...,https://nypost.com/2025/10/21/world-news/trump...,89
325,2025-10-21-17-55-57 +0000,nypost,Hillary mocked after blasting Trump’s White Ho...,https://nypost.com/2025/10/21/us-news/hillary-...,89


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
94,99,2025-10-22-10-00-00 +0000,nypost,Sen. John Kennedy talks Trump’s social media h...,https://nypost.com/2025/10/22/us-news/sen-john...
194,71,2025-10-22-03-11-49 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
31,62,2025-10-22-13-31-00 +0000,wsj,Paul Ingrassia withdrew Tuesday from a Senate ...,https://www.wsj.com/politics/policy/paul-ingra...
100,49,2025-10-22-10-00-00 +0000,nypost,Andrew Cuomo will try one last time to rattle ...,https://nypost.com/2025/10/22/us-news/andrew-c...
9,49,2025-10-22-14-34-06 +0000,nypost,Gruesome new video shows moment former Mass. m...,https://nypost.com/2025/10/22/us-news/ex-mayor...
142,42,2025-10-22-09-00-00 +0000,wsj,The Republican National Committee heads into t...,https://www.wsj.com/politics/policy/democratic...
326,41,2025-10-21-17-44-24 +0000,nypost,Hamas released bodies of two more dead hostage...,https://nypost.com/2025/10/21/world-news/hamas...
219,40,2025-10-22-00-24-39 +0000,nypost,Hunter Biden says watching Obama lead Joe off ...,https://nypost.com/2025/10/21/us-news/hunter-b...
138,37,2025-10-22-09-00-33 +0000,cbc,Children among at least 6 killed in Russian dr...,https://www.cbc.ca/news/world/children-among-a...
238,37,2025-10-21-23-10-23 +0000,nypost,Netflix shares slide on rare earnings miss — s...,https://nypost.com/2025/10/21/media/netflix-sh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
